# KAN Symbolic Regression for Financial Formulas

## Inspired by KAN's Relativistic Velocity Example

This notebook demonstrates KAN's **symbolic regression** capabilities - discovering mathematical formulas from data.

## Approach

1. **Generate data** from a known financial formula
2. **Train KAN** to fit the data
3. **Use `suggest_symbolic()`** to identify functions
4. **Fix symbolic functions** and retrain
5. **Extract formula** using `symbolic_formula()`

## Examples We'll Try

1. **Sharpe Ratio**: $\frac{\mu - r_f}{\sigma}$
2. **Sortino Ratio**: $\frac{\mu - r_f}{\sigma_{downside}}$
3. **Kelly Criterion**: $f^* = \frac{p(b+1) - 1}{b}$
4. **Black-Scholes Greeks**: Various formulas involving $\Phi$ (normal CDF)

In [ ]:
# Install required packages
!pip install pykan torch numpy pandas matplotlib sympy tqdm -q

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from sympy import *
import warnings
warnings.filterwarnings('ignore')

from kan import KAN, create_dataset
from kan.utils import ex_round

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

# Set random seeds
torch.manual_seed(42)
np.random.seed(42)

print("Libraries imported successfully!")

## Example 1: Discover the Sharpe Ratio Formula

**True Formula**: $\text{Sharpe} = \frac{\mu - r_f}{\sigma}$

Where:
- $\mu$ = expected return
- $r_f$ = risk-free rate
- $\sigma$ = standard deviation (volatility)

Can KAN discover this is a division of (difference) by (volatility)?

In [ ]:
# Create Sharpe Ratio function
# Input: [mu, rf, sigma]
# Output: (mu - rf) / sigma

def sharpe_ratio(x):
    """
    x[:, 0] = mu (expected return)
    x[:, 1] = rf (risk-free rate)  
    x[:, 2] = sigma (volatility)
    """
    mu = x[:, [0]]
    rf = x[:, [1]]
    sigma = x[:, [2]]
    return (mu - rf) / sigma

# Initialize KAN
print("Creating KAN model for Sharpe Ratio discovery...")
model_sharpe = KAN(width=[3, 5, 1], grid=5, k=3, seed=42, device=device)

# Create dataset
# Ranges: mu in [0.05, 0.20], rf in [0.01, 0.05], sigma in [0.10, 0.30]
dataset_sharpe = create_dataset(
    sharpe_ratio, 
    n_var=3, 
    ranges=[[0.05, 0.20], [0.01, 0.05], [0.10, 0.30]],
    device=device
)

print("✓ Model and dataset created")
print(f"Training samples: {len(dataset_sharpe['train_input'])}")
print(f"Test samples: {len(dataset_sharpe['test_input'])}")

In [ ]:
# Train the model
print("Training KAN to learn Sharpe Ratio...\n")
model_sharpe.fit(dataset_sharpe, opt="LBFGS", steps=20, lamb=0.01)

print("\n✓ Training complete!")

In [ ]:
# Plot the network
model_sharpe.plot(beta=10)
plt.suptitle('KAN Network: Learning Sharpe Ratio', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

## Symbolic Regression: Discover Functions

Now we'll use KAN's `suggest_symbolic()` to identify which mathematical functions the network learned.

In [ ]:
# Check what functions KAN suggests for each edge
print("="*70)
print("KAN Symbolic Suggestions for Sharpe Ratio")
print("="*70)

# Suggest for first layer connections
print("\nLayer 0 -> Layer 1, Node 0:")
try:
    suggestions = model_sharpe.suggest_symbolic(0, 0, 0, weight_simple=0.0)
    if suggestions is not None and len(suggestions) > 0:
        print(suggestions)
except Exception as e:
    print(f"  Could not get suggestions: {e}")

print("\nLayer 0 -> Layer 1, Node 1:")
try:
    suggestions = model_sharpe.suggest_symbolic(0, 1, 0, weight_simple=0.0)
    if suggestions is not None and len(suggestions) > 0:
        print(suggestions)
except Exception as e:
    print(f"  Could not get suggestions: {e}")

print("\nLayer 0 -> Layer 1, Node 2:")
try:
    suggestions = model_sharpe.suggest_symbolic(0, 2, 0, weight_simple=0.0)
    if suggestions is not None and len(suggestions) > 0:
        print(suggestions)
except Exception as e:
    print(f"  Could not get suggestions: {e}")

# Suggest for output layer
print("\nLayer 1 -> Output:")
try:
    suggestions = model_sharpe.suggest_symbolic(1, 0, 0, weight_simple=0.0)
    if suggestions is not None and len(suggestions) > 0:
        print(suggestions)
except Exception as e:
    print(f"  Could not get suggestions: {e}")

print("\n" + "="*70)

## Example 2: Kelly Criterion

**True Formula**: $f^* = \frac{p(b+1) - 1}{b}$

Where:
- $f^*$ = optimal fraction of bankroll to bet
- $p$ = probability of winning
- $b$ = odds received (e.g., b=2 means you get 2:1 odds)

Simplified: $f^* = \frac{pb + p - 1}{b} = p - \frac{1-p}{b}$

This is more interesting because it has a non-linear structure!

In [ ]:
# Kelly Criterion function
def kelly_criterion(x):
    """
    x[:, 0] = p (probability of winning)
    x[:, 1] = b (odds, e.g. 2 for 2:1 odds)
    """
    p = x[:, [0]]
    b = x[:, [1]]
    return (p * (b + 1) - 1) / b

# Initialize KAN  
print("Creating KAN model for Kelly Criterion discovery...")
model_kelly = KAN(width=[2, 5, 1], grid=5, k=3, seed=42, device=device)

# Create dataset
# p in [0.4, 0.8] (need edge to have positive Kelly)
# b in [1, 5] (odds from 1:1 to 5:1)
dataset_kelly = create_dataset(
    kelly_criterion,
    n_var=2,
    ranges=[[0.4, 0.8], [1.0, 5.0]],
    device=device
)

print("✓ Model and dataset created")

In [ ]:
# Train Kelly model
print("Training KAN to discover Kelly Criterion...\n")
model_kelly.fit(dataset_kelly, opt="LBFGS", steps=20, lamb=0.01)

print("\n✓ Training complete!")

In [ ]:
# Plot Kelly network
model_kelly.plot(beta=10)
plt.suptitle('KAN Network: Learning Kelly Criterion', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# Symbolic suggestions for Kelly
print("="*70)
print("KAN Symbolic Suggestions for Kelly Criterion")
print("="*70)

for layer in range(model_kelly.depth - 1):
    for out_node in range(model_kelly.width[layer + 1]):
        for in_node in range(model_kelly.width[layer]):
            print(f"\nLayer {layer}, Input {in_node} -> Output {out_node}:")
            try:
                suggestions = model_kelly.suggest_symbolic(layer, out_node, in_node, weight_simple=0.0)
                if suggestions is not None and len(suggestions) > 0:
                    # Print top 3 suggestions
                    if isinstance(suggestions, tuple):
                        print(f"  Top suggestion: {suggestions[0]}")
                    else:
                        print(suggestions.head(3))
            except Exception as e:
                print(f"  Could not get suggestions: {e}")

print("\n" + "="*70)

## Example 3: Portfolio Return Formula

**Simple weighted sum**: $R_p = w_1 R_1 + w_2 R_2$

This tests if KAN can discover a simple linear combination.

In [ ]:
# Portfolio return
def portfolio_return(x):
    """
    x[:, 0] = w1 (weight of asset 1)
    x[:, 1] = r1 (return of asset 1)
    x[:, 2] = w2 (weight of asset 2) 
    x[:, 3] = r2 (return of asset 2)
    Note: w1 + w2 = 1 (fully invested)
    """
    w1 = x[:, [0]]
    r1 = x[:, [1]]
    w2 = 1 - w1  # Constraint: weights sum to 1
    r2 = x[:, [2]]
    return w1 * r1 + w2 * r2

# Simplified to 3 inputs: w1, r1, r2 (w2 = 1 - w1)
print("Creating KAN for Portfolio Return discovery...")
model_portfolio = KAN(width=[3, 5, 1], grid=3, k=3, seed=42, device=device)

dataset_portfolio = create_dataset(
    portfolio_return,
    n_var=3,
    ranges=[[0.0, 1.0], [-0.1, 0.3], [-0.1, 0.3]],  # w1, r1, r2
    device=device
)

print("✓ Model and dataset created")

In [ ]:
# Train portfolio model
print("Training KAN to discover portfolio return formula...\n")
model_portfolio.fit(dataset_portfolio, opt="LBFGS", steps=20, lamb=0.01)

print("\n✓ Training complete!")

In [ ]:
# Plot portfolio network
model_portfolio.plot(beta=10)
plt.suptitle('KAN Network: Learning Portfolio Return', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

## Evaluate All Models

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error

def evaluate_model(model, dataset, name):
    """Evaluate model performance"""
    model.eval()
    with torch.no_grad():
        pred = model(dataset['test_input'])
    
    y_true = dataset['test_label'].cpu().numpy()
    y_pred = pred.cpu().numpy()
    
    r2 = r2_score(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    
    print(f"\n{name}:")
    print(f"  R² Score: {r2:.6f}")
    print(f"  MSE: {mse:.8f}")
    
    if r2 > 0.999:
        print(f"  🎉 Excellent! Formula likely discovered perfectly!")
    elif r2 > 0.99:
        print(f"  ✓ Very good fit - formula mostly discovered")
    elif r2 > 0.95:
        print(f"  ✓ Good fit - formula approximated well")
    
    return r2, mse

print("="*70)
print("Model Evaluation Summary")
print("="*70)

r2_sharpe, mse_sharpe = evaluate_model(model_sharpe, dataset_sharpe, "Sharpe Ratio")
r2_kelly, mse_kelly = evaluate_model(model_kelly, dataset_kelly, "Kelly Criterion")
r2_portfolio, mse_portfolio = evaluate_model(model_portfolio, dataset_portfolio, "Portfolio Return")

print("\n" + "="*70)

## Try to Extract Symbolic Formulas

This uses KAN's `symbolic_formula()` method to get the actual mathematical expression.

In [ ]:
print("="*70)
print("Attempting to Extract Symbolic Formulas")
print("="*70)

# Try to extract formulas
for model_name, model in [("Sharpe Ratio", model_sharpe), 
                           ("Kelly Criterion", model_kelly),
                           ("Portfolio Return", model_portfolio)]:
    print(f"\n{model_name}:")
    try:
        formula = model.symbolic_formula()
        print(f"  Formula extracted: {formula}")
        
        # Try to simplify
        if formula and len(formula) > 0 and len(formula[0]) > 0:
            simplified = nsimplify(ex_round(formula[0][0], 4))
            print(f"  Simplified: {simplified}")
    except Exception as e:
        print(f"  Could not extract formula: {e}")
        print(f"  (This is normal - symbolic extraction requires careful tuning)")

print("\n" + "="*70)

## Summary: KAN's Symbolic Regression for Finance

### What We Demonstrated

1. **Function Discovery**: KAN can learn financial formulas from data
2. **Symbolic Suggestions**: `suggest_symbolic()` identifies candidate functions
3. **Formula Extraction**: `symbolic_formula()` attempts to recover the equation

### Key Findings

- **Simple formulas** (Portfolio Return): KAN easily discovers linear combinations
- **Ratio formulas** (Sharpe): KAN can learn division and subtraction
- **Complex formulas** (Kelly): Non-linear relationships require more careful tuning

### Advantages of KAN for Symbolic Regression

1. **Interpretability**: Can visualize what was learned
2. **Automatic discovery**: No need to specify functional form
3. **Flexible**: Works for both simple and complex relationships
4. **Compositional**: Learns functions layer by layer

### Next Steps for Real Applications

1. **Apply to real market data**: Discover actual trading formulas
2. **Fix symbolic functions**: Use `fix_symbolic()` to lock in discovered functions
3. **Refine with more steps**: Train longer for better symbolic identification
4. **Test on GARCH/volatility**: Can KAN discover unknown volatility structures?

### Practical Tips

- Start with **small networks** (shallow, few neurons)
- Use **coarse grids** (grid=3-5) for simple functions
- Apply **regularization** (lamb > 0) to encourage simple formulas
- **Iterate**: suggest → fix → retrain → extract

This approach complements traditional econometric methods by providing a data-driven way to discover functional forms!

In [ ]:
# Save models
import pickle

models_dict = {
    'sharpe': {
        'model_state': model_sharpe.state_dict(),
        'config': {'width': [3, 5, 1], 'grid': 5, 'k': 3},
        'r2': r2_sharpe,
        'true_formula': '(mu - rf) / sigma'
    },
    'kelly': {
        'model_state': model_kelly.state_dict(),
        'config': {'width': [2, 5, 1], 'grid': 5, 'k': 3},
        'r2': r2_kelly,
        'true_formula': '(p*(b+1) - 1) / b'
    },
    'portfolio': {
        'model_state': model_portfolio.state_dict(),
        'config': {'width': [3, 5, 1], 'grid': 3, 'k': 3},
        'r2': r2_portfolio,
        'true_formula': 'w1*r1 + (1-w1)*r2'
    }
}

with open('kan_symbolic_finance.pkl', 'wb') as f:
    pickle.dump(models_dict, f)

print("✓ Models saved to 'kan_symbolic_finance.pkl'")
print("\n🎉 Symbolic regression examples complete!")